In [8]:
from collections import namedtuple
from queue import SimpleQueue, LifoQueue
from itertools import product
from pprint import pprint

I define the state how (ce_pizzeria,ds_pizzeria,ce_pub,ds_pub,bike). Where ce_pizzeria and ce_pub indicate the number of Computer Engineer in the pizzeria and pub, ds_pizzeria and ds_pub indicate the number of Data Scientist in the same two places, and bike indicates the position of bike ('pizzeria' or 'pub).

In [9]:
State = namedtuple(
    "State",
    [
        "ce_pizzeria",
        "ds_pizzeria",
        "ce_pub",
        "ds_pub",
        "bike",
    ],
)

In [10]:
NUM_FRIENDS = 6
BIKE_CAPACITY = 3

I define a function to check if the state is a goal or not

In [11]:
def goal_check(state):
    return (
        state.ce_pizzeria == 0
        and state.ds_pizzeria == 0
        and state.ce_pub == NUM_FRIENDS // 2
        and state.ds_pub == NUM_FRIENDS // 2
    )

I define a function to verify if the state is a valid state or not, considering that all the value have to be positive and count(ce) <= count (ds) or count(ds) == 0

In [12]:
def valid_state(state):
    return (
        state.ce_pizzeria >= 0
        and state.ds_pizzeria >= 0
        and state.ce_pub >= 0
        and state.ds_pub >= 0
        and (state.ce_pizzeria <= state.ds_pizzeria or state.ds_pizzeria == 0)
        and (state.ce_pub <= state.ds_pub or state.ds_pub == 0)
    )

I define all the possible moves with move = (number_of_ce_to_move,number_of_ds_to_move) 

In [13]:
Move = namedtuple("Move", ["ce_in_bike", "ds_in_bike"])

In [14]:
moves = [
    Move(ce, ds)
    for ce, ds in product(range(BIKE_CAPACITY + 1), repeat=2)
    if ce + ds != 0 and ce + ds <= BIKE_CAPACITY
]

In [15]:
def apply_move(state, move):
    if state.bike == "pizzeria":
        return State(
            state.ce_pizzeria - move.ce_in_bike,
            state.ds_pizzeria - move.ds_in_bike,
            state.ce_pub + move.ce_in_bike,
            state.ds_pub + move.ds_in_bike,
            "pub",
        )
    return State(
        state.ce_pizzeria + move.ce_in_bike,
        state.ds_pizzeria + move.ds_in_bike,
        state.ce_pub - move.ce_in_bike,
        state.ds_pub - move.ds_in_bike,
        "pizzeria",
    )

In [16]:
def search(frontier):
    current_path = frontier.get()
    current_state = current_path[-1]
    steps = 0
    while not goal_check(current_state):
        steps += 1
        for move in moves:
            new_state = apply_move(current_state, move)

            if valid_state(new_state) and not new_state in current_path:
                frontier.put(current_path + [new_state])

        current_path = frontier.get()
        current_state = current_path[-1]

    print(f"solved in {steps}")
    return current_path

In [21]:
fifo = SimpleQueue()
lifo = LifoQueue()

fifo.put(
    [
        State(
            NUM_FRIENDS // 2,
            NUM_FRIENDS // 2,
            0,
            0,
            "pizzeria",
        )
    ]
)
lifo.put(
    [
        State(
            NUM_FRIENDS // 2,
            NUM_FRIENDS // 2,
            0,
            0,
            "pizzeria",
        )
    ]
)

path_goal_fifo = search(fifo)
path_goal_lifo = search(lifo)

solved in 47
solved in 7


In [22]:
path_goal_fifo

[State(ce_pizzeria=3, ds_pizzeria=3, ce_pub=0, ds_pub=0, bike='pizzeria'),
 State(ce_pizzeria=2, ds_pizzeria=2, ce_pub=1, ds_pub=1, bike='pub'),
 State(ce_pizzeria=2, ds_pizzeria=3, ce_pub=1, ds_pub=0, bike='pizzeria'),
 State(ce_pizzeria=2, ds_pizzeria=0, ce_pub=1, ds_pub=3, bike='pub'),
 State(ce_pizzeria=3, ds_pizzeria=0, ce_pub=0, ds_pub=3, bike='pizzeria'),
 State(ce_pizzeria=0, ds_pizzeria=0, ce_pub=3, ds_pub=3, bike='pub')]

In [23]:
path_goal_lifo

[State(ce_pizzeria=3, ds_pizzeria=3, ce_pub=0, ds_pub=0, bike='pizzeria'),
 State(ce_pizzeria=0, ds_pizzeria=3, ce_pub=3, ds_pub=0, bike='pub'),
 State(ce_pizzeria=2, ds_pizzeria=3, ce_pub=1, ds_pub=0, bike='pizzeria'),
 State(ce_pizzeria=1, ds_pizzeria=1, ce_pub=2, ds_pub=2, bike='pub'),
 State(ce_pizzeria=2, ds_pizzeria=2, ce_pub=1, ds_pub=1, bike='pizzeria'),
 State(ce_pizzeria=1, ds_pizzeria=0, ce_pub=2, ds_pub=3, bike='pub'),
 State(ce_pizzeria=3, ds_pizzeria=0, ce_pub=0, ds_pub=3, bike='pizzeria'),
 State(ce_pizzeria=0, ds_pizzeria=0, ce_pub=3, ds_pub=3, bike='pub')]